In [7]:
import numpy as np

In [8]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [20,20]


def fix_scaling(ax=None):
    if not ax:
        xlim = plt.xlim()
        ylim = plt.ylim()
        d1 = xlim[1] - xlim[0]
        d2 = ylim[1] - ylim[0]
        if d1 > d2:
            plt.ylim((ylim[0] - (d1-d2) / 2, ylim[1] + (d1-d2) / 2))
        else:
            plt.xlim((xlim[0] + (d1-d2) / 2, xlim[1] - (d1-d2) / 2))
    else:
        xlim = ax.get_xlim()
        ylim = ax.get_ylim()
        d1 = xlim[1] - xlim[0]
        d2 = ylim[1] - ylim[0]
        if d1 > d2:
            ax.set_ylim((ylim[0] - (d1-d2) / 2, ylim[1] + (d1-d2) / 2))
        else:
            ax.set_xlim((xlim[0] + (d1-d2) / 2, xlim[1] - (d1-d2) / 2))

In [10]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML


def animate_trajectory(trajectories, n, f, x_min):
    fig, ax = plt.subplots()
    colors = ['blue', 'orange']
    
    def step(t):
        ax.cla()
        ax.plot([x_min[0]], [x_min[1]], 'o', color='green')
        # Level contours
        delta = 0.025
        x = np.arange(-1, 1, delta)
        y = np.arange(-1, 1, delta)
        X, Y = np.meshgrid(x, y)
        Z = np.zeros_like(X)
        # print(X.shape, Y.shape)
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                Z[i][j] = f([X[i][j], Y[i][j]])
        CS = ax.contour(X, Y, Z, [3.0, 4.0, 5.0], colors=['blue', 'purple', 'red'])

        for traj, color in zip(trajectories, colors):
            ax.plot([u[0] for u in traj[0][:t]], [u[1] for u in traj[0][:t]], color=color)
            ax.plot([u[0] for u in traj[0][:t]], [u[1] for u in traj[0][:t]], 'o', color=color)

        fix_scaling(ax)
        ax.axis('off')

    plt.close()
    return FuncAnimation(fig, step, frames=range(n), interval=600)

In [11]:
NUMBER_OF_STEPS = 9000
PRINT_EACH = 300

In [12]:
from math import exp


def f(x):
    return exp(x[0] + 3 * x[1]) + exp(x[0] - 3 * x[1]) + exp(-x[0])


def f_grad(x):
    return np.array([exp(x[0] + 3 * x[1]) + exp(x[0] - 3 * x[1]) - exp(-x[0]),
                     3 * exp(x[0] + 3 * x[1]) - 3 * exp(x[0] - 3 * x[1])])


def f_grad_l():
    eps = 0.1
    C = 3
    L = eps
    delta = 0.001
    X = np.arange(-1, 1 + delta, delta)

    for i in range(len(X) - 1):
        for y in X:
            x1 = [X[i], y]
            x2 = [X[i + 1], y]
            L = max(L, np.linalg.norm(f_grad(x1) - f_grad(x2)) / delta)
            x1 = [y, X[i]]
            x2 = [y, X[i + 1]]
            L = max(L, np.linalg.norm(f_grad(x1) - f_grad(x2)) / delta)

    return C * L

In [13]:
M = f_grad_l()

In [14]:
def optimal_step(x_start):
    alpha = 1.0 / M
    traj_opt_step = [x_start.copy()]
    cur_x = x_start.copy()
    for i in range(NUMBER_OF_STEPS):
        cur_x = cur_x - alpha * f_grad(cur_x)
        if i % PRINT_EACH == 0:
            traj_opt_step.append(cur_x.copy())

    print(traj_opt_step[-1])
    return [traj_opt_step, 'Optimal step']

In [15]:
def nesterov(x_start):
    traj_nesterov = [x_start.copy()]
    cur_x = x_start.copy()
    alpha = 1.0 / (M + 1.0)
    cur_y = x_start.copy()

    for i in range(NUMBER_OF_STEPS):
        prev_x = cur_x
        cur_x = cur_y - 1.0 / M * f_grad(cur_y)
        prev_alpha = alpha
        alpha = max(np.roots([1, alpha ** 2, -alpha ** 2]))
        beta = prev_alpha * (1 - prev_alpha) / (prev_alpha ** 2 + alpha)
        cur_y = cur_x + beta * (cur_x - prev_x)
        if i % PRINT_EACH == 0:
            traj_nesterov.append(cur_x.copy())

    print(traj_nesterov[-1])
    return [traj_nesterov, 'Nesterov']

In [16]:
x_start = np.array([0.9, 0.7])
trajectories = []
trajectories.append(optimal_step(x_start))
trajectories.append(nesterov(x_start))

[ -3.46573455e-01   9.18596275e-18]


[ -3.46633675e-01  -1.16026822e-17]


In [17]:
from math import log

base_animation = animate_trajectory(trajectories, NUMBER_OF_STEPS // PRINT_EACH, f, np.array([-log(2) / 2, 0]))
HTML(base_animation.to_html5_video())

<video width="1440" height="1440" controls autoplay loop>
  <source type="video/mp4" src="data:video/mp4;base64,AAAAHGZ0eXBNNFYgAAACAGlzb21pc28yYXZjMQAAAAhmcmVlAAEdW21kYXQAAAKtBgX//6ncRem9
5tlIt5Ys2CDZI+7veDI2NCAtIGNvcmUgMTU3IHIyOTM1IDU0NWRlMmYgLSBILjI2NC9NUEVHLTQg
QVZDIGNvZGVjIC0gQ29weWxlZnQgMjAwMy0yMDE4IC0gaHR0cDovL3d3dy52aWRlb2xhbi5vcmcv
eDI2NC5odG1sIC0gb3B0aW9uczogY2FiYWM9MSByZWY9MyBkZWJsb2NrPTE6MDowIGFuYWx5c2U9
MHgzOjB4MTEzIG1lPWhleCBzdWJtZT03IHBzeT0xIHBzeV9yZD0xLjAwOjAuMDAgbWl4ZWRfcmVm
PTEgbWVfcmFuZ2U9MTYgY2hyb21hX21lPTEgdHJlbGxpcz0xIDh4OGRjdD0xIGNxbT0wIGRlYWR6
b25lPTIxLDExIGZhc3RfcHNraXA9MSBjaHJvbWFfcXBfb2Zmc2V0PS0yIHRocmVhZHM9NiBsb29r
YWhlYWRfdGhyZWFkcz0xIHNsaWNlZF90aHJlYWRzPTAgbnI9MCBkZWNpbWF0ZT0xIGludGVybGFj
ZWQ9MCBibHVyYXlfY29tcGF0PTAgY29uc3RyYWluZWRfaW50cmE9MCBiZnJhbWVzPTMgYl9weXJh
bWlkPTIgYl9hZGFwdD0xIGJfYmlhcz0wIGRpcmVjdD0xIHdlaWdodGI9MSBvcGVuX2dvcD0wIHdl
aWdodHA9MiBrZXlpbnQ9MjUwIGtleWludF9taW49MSBzY2VuZWN1dD00MCBpbnRyYV9yZWZyZXNo
PTAgcmNfbG9va2FoZWFkPTQwIHJjPWNyZiBtYnRyZWU9MSBjcmY9MjMuMCBxY29tcD0wLjYwIHFw
bWluPTAgcXBtYXg9NjkgcXBzdGVwPTQgaXBfcmF0aW89MS40MCBhcT0xOjEuMDAAgAAAeWlliIQA
F//+99S3zLLtU2+2C6j3op4mX0N1JQGblsTtOoAAAAMAAAMAAAMAAAMAO+aFu2OTV/QdwkAAAAMA
ABbAAAI8AACIgAAu4AAWIAAL8AAKOAAIUAAGkAAHiAAHCAAIgAANEAARAAADAAADAAADAAADAAAD
AAADAAADAAADAAADAAADAAADATOj/8S/xKwAm0FwGrSFjRW1wbQqjYLqvNnJta0LqwmglasL7UWv
ZR/vMvmdoKKQ78eUajtt62lxiQu9GB2P/Rpro4d/VUI3KNGfXxOh7p6XM8CsC7743uLt2Okk5HnU
NaBv8HIjCJxEyMWkQcwshuR+/2Nx6F8CcEY1MmrUB7JQtYs36mFvL8QDcvscYr7ciIy8bgA7c9Fr
XmLFRo8xHr3Q20HZKEEFz7Cehc+P52SswJI1K24z4KLGujipdiWRansuAqUjv2cZglHRCRolZxMy
u03nd48UXgS/HQ6IvPAw8Wr+D2dQecQBuGTcP8ONVfme+ruCV3LjTLk1A58ueeLkAe74vMcB9shl
PQCBUGqSb7Lp9b5QTDFKxAkQWbPqskx1I3qkkZo7mHI+pB++vRWJnKzOezoaBcbIeXERXVzoJ5fs
KsTt/s2KI4ZuDMd/fvZt1PzBwstKfNv1sVNXsNb7XNIEetwwu/4U+ym1imz2YKBGEg8afZwEsoQg
f9Kmob1trKDKfKGAYU0EZ6XRRQM1BSbE9T1XBO/b/48UCw2GarlAf+Sl/skhNPriMTg0I/tdX+eR
T7Z9wCw+aJIF878CO8d9eeq1ha9dOcNVDobu9BEzR/E0lwPuKYUSelLFdhnjsiGrXFDyUg3DEda2
xckOqLrGed00ScxOh8y2d4VKgNgdPZJSW1lciOl8kvfxyWxeAW7hwTNnlSb8zKvg+6lQrIR1b5W3
tTGvkSrmihdMormhGVYONQPb5R6Yhly138fnj0RDWiSNRoOCmWJFAXD67NasXPfIdjMorvEdkPmx
iMsW/9wcehqklABksK0xJWdzcA9Pm4PLu/ni+sbBAEYhKVKVOa6P5BJP/5Jq3ErLIuQss7CX54j0
t12vWuslfD0kiKtICmJk3sA0BEFCsmKzAx3Bslvnr4pkIAXXliwljp7bg3tLHRSEE/isR10cd4wo
q414828CFBn+5z7lJKiwA483AGs5oEB0Bhr/yhUDzqhL9AO1tPgdJR57+xdI7t+Dzit/3rQU628a
ArkN1kmKcT+/CizfAGketxGViIszYttulKhCOaH7dqGjNXmz3podCjkO8GHgHYem2V4JbhcWx8eo
sdiIxekFDCVvRlwhBbNjIBUdgb0+s2RepJCycqe0su+dMNWq87lSJEgD0f0Ypmx56Nryc89pHEEx
Ytl7SAAEdHmoyUTfnUAVz6ST31bmxszBuRlKl6KF5d2p5I33l2ydbqPPvc3op1zSaBJ9qQCEfKDN
GO21RONQWHhRmOF/3rcDxfvZC9E1JbUmW+teAuGKF0A9NqNlPpqraZZkycU26usLjNJSzh1ME1l4
qy9pePT8LuNiHQBTnpnYT/uigTOzIw+XcqjWCkSzyz9G+RNI5mEgxScGqGyyJqHmPM8XLLfmGgxZ
JU1RwQOiNYPx45sVh0czrQSG82gilSHEmrpaaHok3mrZfd9Q6iotAEzBkRC0T6JpwZlC4b4j/WPx
N9H2toRPqC+yjfmgghBY1xz7t0kYVTrq5sZhxNOTUunmfQBwMaqGF/8qq4+fIHRarq3WeImRiqS5
wOovTWMIUtEuBmKZczmwaomBuDeX3xxVsQGfAYw97zTz4nAm5ePFfxvhyYJ5oqgTFrOWSddQdIOL
0u63RqGe0lCYaZWozqPkCgF5kFUt/mj4rQpYEC3BwrS6z+nM958nviimRnn+mgYd9+sJP1iX55WF
KRLOPNKd3dbL9etj5Nfi24eVYw3TOTfEGYIGWnx5U1rsDhlAyYCir7Fqmxwgc2KIkGIkW5wKTPdI
Zr0cUQU2c1HhdECtNSMF5PRbb8bcUgMmLTwUiXewzIAAAAZxcqb909saDe1uVhF7kCtu1sFtZhK/
2ximkOnNdaVVB+GWd39l3ziVxn1dFvOxZ7IfW1LmKn31dCEbSJHm3fO7PmG+rZA7uvuuzz9IVRGu
pfMLn7CFi8VZ7PIrpIhzR2WCyK3xx8Ir2b6V4M6ma4/XUkHO35Y1t940q5FaS/U60mfPPP1ZMKUk
xDfc06IVmgf9ZJNuzfjGt6sob7gnvnufvI2+CndGpZLFVXsDM+D1k9Fyq7aamx5kLTiY5XsFtO6c
FWuRuQ4Q/gC6NP0HhAcc97WuvB/wJ7v+V7ZS/EKgn5H9wx36t2oerog3bhj7DHWPmTaxow5j2Snj
l3WrfSsCRztNTY8VmFOQbOfWZIEFy2hWdJmLr0JdvdN441zYqLPd/r1yKosOm/xZTxvcfa26Ir4u
mfrx2KhhYuXE0DNigxKMdrAfOoTwEbFIXPwRgM4ssFRHAb1uMCV0hWMKZjKp+rrwh7jPxWWs0sSh
p9CyQ7Y7hj7PWYQqzKLttgye58oVKxV1dxQ+10XSGE2KGCELlayUK+IS9zHRLw8UVX6m2Pl3dmna
9qAqJA0crqubmEc/XZrqEOoo+988/TXFBwbBplGeCsnBDV3+2uade+i1OrR9xRtPP14h4MheWbR1
S+FC3JIxwnjsBQddAAr3r20fl6Abjfth0FAyLTQEXqY920wJNFEjAcnASnWL12EpTqf4eUe7asJt
Kk7WxRwMkk2tLsOegWLukHX4diA6JGg4WBmL7pVMILo/bnw9J7cmdtI9e8rlcmTqRXkm4iD0yqhE
Jk6HoesWkcKAisUtoRNRwr64iTmQEOVuHYvcce